In [2]:
!pip install git+https://github.com/capeanalytics/data_store > /dev/null

  Running command git clone -q https://github.com/capeanalytics/data_store /tmp/pip-req-build-nr7j_1_g
  distutils: /tmp/pip-build-env-rw1yxvbr/normal/lib/python3.10/site-packages
  sysconfig: /tmp/pip-build-env-rw1yxvbr/normal/local/lib/python3.10/dist-packages
  distutils: /tmp/pip-build-env-rw1yxvbr/normal/lib/python3.10/site-packages
  sysconfig: /tmp/pip-build-env-rw1yxvbr/normal/local/lib/python3.10/dist-packages
  user = False
  home = None
  root = None
  prefix = '/tmp/pip-build-env-rw1yxvbr/normal'
  distutils: /tmp/pip-build-env-rw1yxvbr/overlay/lib/python3.10/site-packages
  sysconfig: /tmp/pip-build-env-rw1yxvbr/overlay/local/lib/python3.10/dist-packages
  distutils: /tmp/pip-build-env-rw1yxvbr/overlay/lib/python3.10/site-packages
  sysconfig: /tmp/pip-build-env-rw1yxvbr/overlay/local/lib/python3.10/dist-packages
  user = False
  home = None
  root = None
  prefix = '/tmp/pip-build-env-rw1yxvbr/overlay'
You should consider upgrading via the '/usr/bin/python -m pip install 

In [3]:
from data_store import DataStore
from data_store.data_containers import Experiments, Dataset
from data_store.storage import GCSStorage
import glob
import os
import yaml
import pandas as pd
import glob

import nest_asyncio
nest_asyncio.apply()

/usr/local/lib/python3.10/dist-packages/data_store/storage/gcs_utils/bulk_download.py:12: UserWarning: Running in jupyter notebook. Using threads instead of asyncio.If you need to download a large number of files consider running the download in a script.
  warnings.warn(


In [4]:
# %%


gs = GCSStorage(
    # endpoint="http://0.0.0.0:4443", 
    path=f"gs://cape-ml-projects-data/data_stores"
)
ds = DataStore(name="dinov2", storage=gs)

Populating queue...
Download started...
Queue fully populated. Found 1 files to download
Processed 0 files in 5.2 s. In queue: 225
Processed 1 files in 10.2 s. In queue: 186
Downloaded 1 files in 13.9 s


In [5]:
ds['experiments/lightly'] = Experiments(description="Experiments using the lightly library")

Starting upload of 1 files
Finished 1 uploads in 15.0 s


In [6]:
exp = ds['experiments/lightly']

In [9]:
meta = dict(
    description = "VicRegL on the RCR 2M dataset",
    max_epochs = 50,
    warmup = 10,  # use 20 if epochs is 800 or 10 if epochs is 200
    knn_k = 200,
    knn_t = 0.1,
    classes = 6,
    input_size = 224,

    # Set to True to enable Distributed Data Parallel training.
    distributed = True,

    # Set to True to enable Synchronized Batch Norm (requires distributed=True).
    # If enabled the batch norm is calculated over all gpus, otherwise the batch
    # norm is only calculated from samples on the same gpu.
    sync_batchnorm = True,

    # Set to True to gather features from all gpus before calculating
    # the loss (requires distributed=True).
    # If enabled then the loss on every gpu is calculated with features from all
    # gpus, otherwise only features from the same gpu are used.
    gather_distributed = True,

    # benchmark
    n_runs = 1,  # optional, increase to create multiple runs and report mean + std
    batch_size = 256 * 4,
    lr_factor = 256 * 4 / 256  # scales the learning rate linearly with batch size
)

In [10]:
e = exp.create(experiment_id="NO-ID-1", **meta)

In [11]:
e.upload("benchmark.py")

Starting upload of 1 files
Finished 1 uploads in 15.0 s


In [13]:
e.upload(glob.glob("*.ckpt"), root_path=os.getcwd())

Starting upload of 2 files
Finished 2 uploads in 15.0 s


In [15]:
ds['rcr_2M_imagenet'] = Dataset(description="RCR 2M dataset in Imagenet format")

Starting upload of 1 files
Finished 1 uploads in 15.0 s


In [18]:
files = [x for x in glob.glob("/data/2m/**/*", recursive=True) if os.path.isfile(x)]

In [19]:
len(files)

2197171

In [ ]:
ds['rcr_2M_imagenet'].upload(files, root_path="/data/2m")

Starting upload of 2197171 files
Completed 0/2197171 uploads in 25.2 s
Completed 4586/2197171 uploads in 40.2 s
Completed 9450/2197171 uploads in 55.2 s
Completed 14635/2197171 uploads in 70.2 s
Completed 19677/2197171 uploads in 85.2 s
Upload failed with: 503, message='Service Unavailable: {\n  "error": {\n    "code": 503,\n    "message": "We encountered an internal error. Please try again.",\n    "errors": [\n      {\n        "message": "We encountered an internal error. Please try again.",\n        "domain": "global",\n        "reason": "backendError"\n      }\n    ]\n  }\n}\n', url=URL('https://www.googleapis.com/upload/storage/v1/b/cape-ml-projects-data/o?name=data_stores/dinov2/rcr_2M_imagenet/train/n00000002/n00000002_945180.png&uploadType=media'), stopping upload.
Upload failed with exception <class 'aiohttp.client_exceptions.ClientResponseError'>: 503, message='Service Unavailable: {\n  "error": {\n    "code": 503,\n    "message": "We encountered an internal error. Please try 

In [4]:
ds['experiments/a100x4'] = Experiments(description="Experiments run on a single A100x4 machine")

Starting upload of 1 files
Finished 1 uploads in 15.0 s


In [5]:
exp = ds['experiments/a100x4']

In [7]:
with open("/data/output/config.yaml") as fp:
    config = yaml.safe_load(fp)

In [12]:
meta = pd.json_normalize(config).iloc[0].to_dict()

In [13]:
e = exp.create(experiment_id="DIN-121", description="Run on the 2m dataset", **meta)

In [21]:
e.upload(["/data/output/config.yaml", "/data/output/training_metrics.json"], root_path="/data/output/")

Starting upload of 2 files
Finished 2 uploads in 15.0 s


In [15]:
e.tree()

data_stores/dinov2/experiments/a100x4/DIN-121/
└── config.yaml


In [18]:
files = [x for x in glob.glob("/data/output/eval/**/*", recursive=True) if os.path.isfile(x)]
e.upload(files, root_path="/data/output")

Starting upload of 10 files
Completed 0/10 uploads in 15.0 s
Finished 10 uploads in 30.1 s


In [22]:
e.tree()

data_stores/dinov2/experiments/a100x4/DIN-121/
├── config.yaml
├── training_metrics.json
└── eval/
    ├── training_112499/
    │   └── teacher_checkpoint.pth
    ├── training_12499/
    │   └── teacher_checkpoint.pth
    ├── training_124999/
    │   └── teacher_checkpoint.pth
    ├── training_24999/
    │   └── teacher_checkpoint.pth
    ├── training_37499/
    │   └── teacher_checkpoint.pth
    ├── training_49999/
    │   └── teacher_checkpoint.pth
    ├── training_62499/
    │   └── teacher_checkpoint.pth
    ├── training_74999/
    │   └── teacher_checkpoint.pth
    ├── training_87499/
    │   └── teacher_checkpoint.pth
    └── training_99999/
        └── teacher_checkpoint.pth
